<a href="https://colab.research.google.com/github/pascalghanimi/Ski-Classification-AI/blob/main/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pickle
from glob import glob
files = glob("*_features.pkl")

with open(files[0], "rb") as f:
  test_file = pickle.load(f)


# features_with_labels beinhaltet fahrstil, schwung_labels, COM_to_ground, knee_angles_right, knee_angles_left, joint_angles, axis_angles, COM_angles
# fahrstil beinhaltet einen einzigen Wert, z.B. EKK -> Wert (einfach ein Wert)
# schwung_labels beinhalten für jeden Frame den Schwung, also Rechts oder Linksschwung (1 Linksschwung, 2 Rechtsschwung) Frame: Wert
# COM_to_ground beinhaltet für jeden Frame den Abstand zwischen COM und Boden Frame: Wert
# knee_angles_right und knee_angles_left beinhalten für jeden Frame die Kniewinkel (also Winkel zwischen Oberschenkel und Unterschenkel) -> Frame: Wert
# joint_angles gibt die Winkel zwischen den Segmenten (z.B. Oberarm, genannt shoulder to ellbow) und den 3 Achsen (x, y und z) -> Frame: { Achse: { Obarm links, Oberschenkel links, etc.}}
# axis_angles gibt die Winkel der Achsen (z.B. Schulterachse, Hüftachse, etc.) zu den 3 Achsen (x, y und z) -> Frame: { Achse: { Schulterachse, Hüftachse }}
# COM_angles gibt Winkel zwischen dem Vektor Boden zu COM und den 3 Achsen (x, y und z) -> Frame: {x-Winkel, y-Winkel, z-Winkel}
print("Fahrstil", test_file["fahrstil"])
print("Schwung Label (rechts oder links)", test_file["schwung_labels"])
print("COM zum Boden Abstand", test_file["COM_to_ground"])
print("Kniewinkel rechts", test_file["knee_angles_right"])
print("Winkel zwischen Segmenten und Achsen", test_file["joint_angles"])
print("Winkel zwischen Körperchsen (z.B. Schulterachse) und Achsen", test_file["axis_angles"])
print("Winkel zwischen Vektor Boden zum COM und den 3 Achsen", test_file["COM_angles"])

Fahrstil CL
Schwung Label (rechts oder links) {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 2, 23: 2, 24: 2, 25: 2, 26: 2, 27: 2, 28: 2, 29: 2, 30: 2, 31: 2, 32: 2, 33: 2, 34: 2, 35: 2, 36: 2, 37: 2, 38: 2, 39: 2, 40: 2, 41: 2, 42: 2, 43: 2, 44: 2, 45: 2, 46: 2, 47: 2, 48: 2, 49: 2, 50: 2, 51: 2, 52: 2, 53: 2, 54: 2, 55: 2, 56: 2, 57: 2, 58: 2, 59: 2, 60: 2, 61: 2, 62: 2, 63: 1, 64: 1, 65: 1, 66: 1, 67: 1, 68: 1, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 1, 75: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 81: 1, 82: 1, 83: 1, 84: 1, 85: 1, 86: 1, 87: 1, 88: 1, 89: 1, 90: 1, 91: 1, 92: 1, 93: 1, 94: 1, 95: 1, 96: 1, 97: 1, 98: 1, 99: 1, 100: 1, 101: 1, 102: 1, 103: 1, 104: 1, 105: 1, 106: 1, 107: 1, 108: 1, 109: 1, 110: 2, 111: 2, 112: 2, 113: 2, 114: 2, 115: 2, 116: 2, 117: 2, 118: 2, 119: 2, 120: 2, 121: 2, 122: 2, 123: 2}
COM zum Boden Abstand {0: np.float32(0.77476716), 1: np.float32(0.77103

In [25]:
for key in test_file["schwung_labels"]:
  if (key == 0):
    print(test_file["COM_to_ground"][key])
    print(test_file["knee_angles_right"][key])
    print(test_file["knee_angles_left"][key])
    for axis in test_file["joint_angles"][key]:
      print(test_file["joint_angles"][key][axis].values())

    for axis in test_file["axis_angles"][key]:
      print(test_file["axis_angles"][key][axis].values())

    print(test_file["COM_angles"][key].values())

0.77476716
146.8749
72.70104
dict_values([np.float64(65.51576979796334), np.float64(125.59187462516205), np.float64(48.236449234851506), np.float64(91.16290779696908), np.float64(41.49285040874689), np.float64(57.192783008204756)])
dict_values([np.float64(40.98451899946306), np.float64(38.0641717641743), np.float64(51.36927327324015), np.float64(73.70713156055584), np.float64(49.90814556006951), np.float64(47.30681191005794)])
dict_values([np.float64(120.55158836546123), np.float64(78.26104540540113), np.float64(114.0910923907794), np.float64(163.66336314624243), np.float64(81.05305970447425), np.float64(60.22176631841171)])
dict_values([np.float64(11.763697131856855), np.float64(22.292308963542837), np.float64(31.549865378570924), np.float64(13.237903629658167)])
dict_values([np.float64(100.03852571204148), np.float64(111.79986966646361), np.float64(78.44394264397089), np.float64(80.84733867953648)])
dict_values([np.float64(96.06989019920216), np.float64(94.43521102643986), np.float64

In [26]:
import numpy as np

# ich speichere die Werte in den Arrays, die y-Werte sind ein einfacher Array der Struktur [1, 2, 1, 1, 2], welcher für jeden Input Vektor den Output (Links- oder Rechtsschwung) ausgibt
# die data beinhaltet Unterarrays, die jeweils einen Wert für jedes Feature beinhalten, jeder Unterarray entspricht einem Frame
#  -> [[Frame 1 Feature 1, Frame 1 Feature 2, Frame 1, Feature 2], [Frame 2 Feature 1, Frame 2 Feature 2...], [ hier Werte dritter Frame alle Feature], ...]

data = []
labels = []
fahrstil_labels = []

MAX_LENGTH = 400

def augment_sequence(seq, noise_std=0.005, jitter_std=0.001, warp_strength=0.1, shift_max=10):
    seq = np.array(seq)

    # 0. Time Shift (zufällige Verschiebung nach vorn oder hinten)
    shift = np.random.randint(-shift_max, shift_max + 1)
    if shift > 0:
        seq = np.pad(seq, ((shift, 0), (0, 0)), mode='constant')[:len(seq)]
    elif shift < 0:
        seq = np.pad(seq, ((0, -shift), (0, 0)), mode='constant')[-shift:]
    # shift == 0 → bleibt unverändert

    # 1. Noise Injection
    seq += np.random.normal(0, noise_std, seq.shape)

    # 2. Jittering (feines zufälliges Zucken)
    seq += np.random.normal(0, jitter_std, seq.shape)

    # 3. Time Warping
    stretch_factor = 1 + np.random.uniform(-warp_strength, warp_strength)
    t_original = np.linspace(0, 1, len(seq))
    t_warped = np.linspace(0, 1, int(len(seq) * stretch_factor))
    warped = np.array([
        np.interp(t_original, np.clip(t_warped, 0, 1),
                  np.interp(t_warped, np.linspace(0, 1, len(seq)), seq[:, i]))
        for i in range(seq.shape[1])
    ]).T

    # Länge anpassen
    warped = warped[:len(seq)] if warped.shape[0] >= len(seq) else np.pad(warped, ((0, len(seq) - warped.shape[0]), (0, 0)))
    return warped



for file in files:
    with open(file, "rb") as f:
        features = pickle.load(f)

    sequence_data = []
    sequence_labels = []

    # Extrahiere Features für jeden Frame
    for frame_idx in features["schwung_labels"]:
        try:
            frame_features = [
                features["COM_to_ground"][frame_idx],
                features["knee_angles_right"][frame_idx],
                features["knee_angles_left"][frame_idx]
            ]

            # Füge joint_angles hinzu
            for axis in features["joint_angles"][frame_idx]:
                frame_features.extend(features["joint_angles"][frame_idx][axis].values())

            # Füge axis_angles hinzu
            for axis in features["axis_angles"][frame_idx]:
                frame_features.extend(features["axis_angles"][frame_idx][axis].values())

            # Füge COM_angles hinzu
            frame_features.extend(features["COM_angles"][frame_idx].values())

            sequence_data.append(frame_features)
            sequence_labels.append(features["schwung_labels"][frame_idx])

        except KeyError as e:
            print(f"Fehler in Datei {file}, Frame {frame_idx}: {str(e)}")
            break

    # Padding für zu kurze Sequenzen
    if len(sequence_data) < MAX_LENGTH:
        pad_length = MAX_LENGTH - len(sequence_data)
        pad_features = [0] * len(sequence_data[0]) if sequence_data else [0] * 27  # Annahme: 27 Features
        sequence_data += [pad_features] * pad_length
        sequence_labels += [0] * pad_length

    data.append(sequence_data)
    labels.append(sequence_labels)

    # Fahrstil vereinheitlichen
    vereinheitlicht = features["fahrstil"].replace("2", "")
    fahrstil_labels.append(vereinheitlicht)



# Konvertiere zu NumPy-Arrays
data = np.array(data, dtype=np.float32)
fahrstil_labels = np.array(fahrstil_labels)

print (data)

[[[  0.77476716 146.8749      72.70104    ... 124.00484    145.82196
    93.03671   ]
  [  0.77103394 142.31094     74.369995   ... 121.83235    147.15121
    97.27488   ]
  [  0.78346187 139.21902     79.39359    ... 119.25487    150.14532
    95.43875   ]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]]

 [[  0.759155   114.05411    131.22078    ...  80.83241    168.92679
    83.842674  ]
  [  0.80178314 123.063774   139.16151    ...  87.216934   171.77281
    82.26397   ]
  [  0.82132035 127.046005   140.98975    ...  92.90956    172.96512
    83.60051   ]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
 

In [27]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from collections import Counter



# ------------------- Datenvorbereitung -------------------
# 1. Label Encoding für Fahrstile
le = LabelEncoder()
fahrstil_encoded = le.fit_transform(fahrstil_labels)

# 2. Daten-Normalisierung
scaler = StandardScaler()
original_shape = data.shape

# Skalierung durchführen
data_normalized = scaler.fit_transform(data.reshape(-1, data.shape[-1]))  # Flatten zu 2D
data_normalized = data_normalized.reshape(original_shape)  # Zurück zu 3D

# 3. Stratifizierte Train/Val/Test Split
X_train, X_temp, Y_train, Y_temp = train_test_split(
    data_normalized,
    fahrstil_encoded,
    test_size=0.3,
    stratify=fahrstil_encoded,
    random_state=42
)

X_val, X_test, Y_val, Y_test = train_test_split(
    X_temp,
    Y_temp,
    test_size=0.5,
    stratify=Y_temp,
    random_state=42
)

# ✅ 👉 HIER Augmentierung NUR für das Trainingsset
X_train_aug = np.array([augment_sequence(x) for x in X_train])
Y_train_aug = Y_train.copy()

label_counts = Counter(Y_train)
median_count = np.median(list(label_counts.values()))

X_aug = []
Y_aug = []

for x, y in zip(X_train, Y_train):

      for _ in range(3):  # 3x Augmentierung pro Sample der Minderheitsklasse
        x_aug = augment_sequence(x)
        X_aug.append(x_aug)
        Y_aug.append(y)


# 4. Kombiniere Original + augmentierte Daten
X_train_combined = np.concatenate([X_train, np.array(X_aug)])
Y_train_combined = np.concatenate([Y_train, np.array(Y_aug)])


# 5. Konvertierung zu PyTorch Tensoren (KORRIGIERT)
X_train = torch.tensor(X_train_combined, dtype=torch.float32)
Y_train = torch.tensor(Y_train_combined, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.long)


# ------------------- DataLoader -------------------
batch_size = 16
train_dataset = TensorDataset(X_train, Y_train)
val_dataset = TensorDataset(X_val, Y_val)
test_dataset = TensorDataset(X_test, Y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [28]:
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

# ------------------- Modellarchitektur -------------------
class SkiSwingLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=0.3
        )
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # *2 wegen bidirectional

    def forward(self, x):
        output, (hidden, _) = self.lstm(x)
        # Kombiniere die letzten Hidden States beider Richtungen
        hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        return self.fc(self.dropout(hidden))

# ------------------- Training Setup -------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SkiSwingLSTM(
    input_size=data.shape[-1],  # Anzahl Features pro Frame
    hidden_size=256,
    num_classes=len(le.classes_)
).to(device)

# Klassen-Gewichte
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(Y_train.numpy()),
    y=Y_train.numpy()
)
weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
loss_fn = nn.CrossEntropyLoss(weight=weights_tensor)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# ------------------- Training -------------------
best_val_accuracy = 0
patience = 50
patience_counter = 0

for epoch in range(300):
    # Trainingsschleife
    model.train()
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = loss_fn(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # Validierungsschleife
    model.eval()
    val_preds = []
    val_true = []
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            preds = model(batch_x.to(device)).argmax(dim=1).cpu()
            val_preds.extend(preds.numpy())
            val_true.extend(batch_y.numpy())

    val_accuracy = np.mean(np.array(val_preds) == np.array(val_true))
    print(f"Epoch {epoch+1:3d} | Val Accuracy: {val_accuracy:.4f}")

    # Early Stopping
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early Stopping nach {epoch+1} Epochen")
            break

# ------------------- Evaluation -------------------
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

all_preds = []
all_true = []
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        preds = model(batch_x.to(device)).argmax(dim=1).cpu()
        all_preds.extend(preds.numpy())
        all_true.extend(batch_y.numpy())

print("\nKlassifikationsbericht:")
print(classification_report(
    all_true,
    all_preds,
    labels=range(len(le.classes_)),
    target_names=le.classes_,
    zero_division=0
))

Epoch   1 | Val Accuracy: 0.2804
Epoch   2 | Val Accuracy: 0.2617
Epoch   3 | Val Accuracy: 0.5888
Epoch   4 | Val Accuracy: 0.4486
Epoch   5 | Val Accuracy: 0.4953
Epoch   6 | Val Accuracy: 0.5981
Epoch   7 | Val Accuracy: 0.4579
Epoch   8 | Val Accuracy: 0.7103
Epoch   9 | Val Accuracy: 0.7103
Epoch  10 | Val Accuracy: 0.7850
Epoch  11 | Val Accuracy: 0.7664
Epoch  12 | Val Accuracy: 0.7570
Epoch  13 | Val Accuracy: 0.7757
Epoch  14 | Val Accuracy: 0.8505
Epoch  15 | Val Accuracy: 0.7477
Epoch  16 | Val Accuracy: 0.8598
Epoch  17 | Val Accuracy: 0.8692
Epoch  18 | Val Accuracy: 0.5888
Epoch  19 | Val Accuracy: 0.7570
Epoch  20 | Val Accuracy: 0.7757
Epoch  21 | Val Accuracy: 0.8224
Epoch  22 | Val Accuracy: 0.8037
Epoch  23 | Val Accuracy: 0.8224
Epoch  24 | Val Accuracy: 0.8037
Epoch  25 | Val Accuracy: 0.7850
Epoch  26 | Val Accuracy: 0.8318
Epoch  27 | Val Accuracy: 0.7383
Epoch  28 | Val Accuracy: 0.7757
Epoch  29 | Val Accuracy: 0.8692
Epoch  30 | Val Accuracy: 0.8692
Epoch  31 

In [29]:
torch.save(model, "ski_schwung_classifier.pt")

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_temp, Y_train, Y_temp = train_test_split(
    data,
    labels,
    test_size=0.3,
    random_state=42,
    shuffle=True
)

X_val, X_test, Y_val, Y_test = train_test_split(
    X_temp,
    Y_temp,
    test_size=0.5,
    random_state=42,
    shuffle=True
)


In [31]:
data = []
labels = []
fahrstil_labels = []

# Fahrstile vorher encodieren
fahrstile_raw = [pickle.load(open(f, "rb"))["fahrstil"] for f in files]
le = LabelEncoder()
le.fit(fahrstile_raw)

for file in files:
    with open(file, "rb") as f:
        features = pickle.load(f)

    sequence_data = []
    sequence_labels = []

    fahrstil_encoded = le.transform([features["fahrstil"]])[0]  # als Zahl

    for frame_idx in features["schwung_labels"]:
        try:
            frame_features = [
                features["COM_to_ground"][frame_idx],
                features["knee_angles_right"][frame_idx],
                features["knee_angles_left"][frame_idx]
            ]

            for axis in features["joint_angles"][frame_idx]:
                frame_features.extend(features["joint_angles"][frame_idx][axis].values())

            for axis in features["axis_angles"][frame_idx]:
                frame_features.extend(features["axis_angles"][frame_idx][axis].values())

            frame_features.extend(features["COM_angles"][frame_idx].values())

            frame_features.append(fahrstil_encoded)  # Fahrstil konstant pro Frame

            sequence_data.append(frame_features)
            sequence_labels.append(features["schwung_labels"][frame_idx])

        except KeyError as e:
            print(f"Fehler in Datei {file}, Frame {frame_idx}: {str(e)}")
            break

    if len(sequence_data) < MAX_LENGTH:
        pad_len = MAX_LENGTH - len(sequence_data)
        pad_feat = [0] * len(sequence_data[0]) if sequence_data else [0] * 28
        sequence_data += [pad_feat] * pad_len
        sequence_labels += [0] * pad_len

    data.append(sequence_data)
    labels.append(sequence_labels)
    fahrstil_labels.append(fahrstil_encoded)


In [32]:
data = np.array(data, dtype=np.float32)
labels = np.array(labels, dtype=np.int64)

X_train, X_temp, Y_train, Y_temp = train_test_split(data, labels, test_size=0.3, random_state=42, shuffle=True)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42, shuffle=True)

X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train, Y_train), batch_size=16, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, Y_val), batch_size=16)
test_loader = DataLoader(TensorDataset(X_val, Y_val), batch_size=16)



In [33]:
class FrameWiseLSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True, bidirectional=True, dropout=0.3)
    self.dropout = nn.Dropout(0.3)
    self.fc = nn.Linear(hidden_size * 2, num_classes)

  def forward(self, x):
    lstm_out, _ = self.lstm(x)
    return self.fc(self.dropout(lstm_out))

device = torch.device("cuda" if torch.cuda.is_available() else "cpi")
model = FrameWiseLSTM(input_size=data.shape[-1], hidden_size=128, num_classes=3).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

best_val_accuracy = 0
patience = 30
patience_counter = 0

for epoch in range(100):
  model.train()
  for batch_x, batch_y in train_loader:
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    optimizer.zero_grad()
    output = model(batch_x)
    output = output.permute(0, 2, 1)
    loss = loss_fn(output, batch_y)
    loss.backward()
    optimizer.step()

  model.eval()
  val_preds = []
  val_true = []
  with torch.no_grad():
    for batch_x, batch_y in val_loader:
      batch_x = batch_x.to(device)
      preds = model(batch_x).argmax(dim=-1).cpu()
      val_preds.append(preds)
      val_true.append(batch_y)




  val_preds = torch.cat(val_preds).flatten().numpy()
  val_true = torch.cat(val_true).flatten().numpy()

  # ❗ Padding rausfiltern (0 = kein echtes Label)
  mask = val_true != 0
  val_accuracy = np.mean(val_preds[mask] == val_true[mask])

  print(f"Epoch {epoch+1:3d} | Val Accuracy: {val_accuracy:.4f}")


  if val_accuracy > best_val_accuracy:
    best_val_accuracy = val_accuracy
    patience_counter = 0
    torch.save(model.state_dict(), "best_model_framewise.pth")
  else:
    patience_counter += 1
    if patience_counter >= patience:
      print(f"Early Stopping at Epoch {epoch + 1}")
      break;


model.load_state_dict(torch.load("best_model_framewise.pth"))
model.eval()
all_preds = []
all_true = []

with torch.no_grad():
  for batch_x, batch_y in test_loader:
    batch_x = batch_x.to(device)
    preds = model(batch_x).argmax(dim=-1).cpu()
    all_preds.append(preds)
    all_true.append(batch_y)

all_preds = torch.cat(all_preds).flatten().numpy()
all_true = torch.cat(all_true).flatten().numpy()

# ❗ Nur echte Labels vergleichen
mask = all_true != 0
print("\nKlassifikationsbericht:")
print(classification_report(all_true[mask], all_preds[mask], digits=3, zero_division=0))

Epoch   1 | Val Accuracy: 0.8299
Epoch   2 | Val Accuracy: 0.8849
Epoch   3 | Val Accuracy: 0.8490
Epoch   4 | Val Accuracy: 0.8777
Epoch   5 | Val Accuracy: 0.9154
Epoch   6 | Val Accuracy: 0.9263
Epoch   7 | Val Accuracy: 0.9232
Epoch   8 | Val Accuracy: 0.9240
Epoch   9 | Val Accuracy: 0.9168
Epoch  10 | Val Accuracy: 0.9269
Epoch  11 | Val Accuracy: 0.9018
Epoch  12 | Val Accuracy: 0.8756
Epoch  13 | Val Accuracy: 0.8809
Epoch  14 | Val Accuracy: 0.8957
Epoch  15 | Val Accuracy: 0.9076
Epoch  16 | Val Accuracy: 0.9349
Epoch  17 | Val Accuracy: 0.9274
Epoch  18 | Val Accuracy: 0.9321
Epoch  19 | Val Accuracy: 0.9125
Epoch  20 | Val Accuracy: 0.9304
Epoch  21 | Val Accuracy: 0.9183
Epoch  22 | Val Accuracy: 0.9368
Epoch  23 | Val Accuracy: 0.9245
Epoch  24 | Val Accuracy: 0.9214
Epoch  25 | Val Accuracy: 0.9296
Epoch  26 | Val Accuracy: 0.9451
Epoch  27 | Val Accuracy: 0.9338
Epoch  28 | Val Accuracy: 0.9462
Epoch  29 | Val Accuracy: 0.9082
Epoch  30 | Val Accuracy: 0.9395
Epoch  31 

In [34]:
torch.save(model, "left_right_classifier.pt")